In [1]:
import re
import os
import sys
from time import time
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy
from tempfile import mkdtemp
from shutil import rmtree
import string  
from bert_serving.client import BertClient

from torch import nn
from skorch import NeuralNetClassifier
from skorch.callbacks import WarmRestartLR, LRScheduler, EarlyStopping
from torch.optim.lr_scheduler import CyclicLR

from sklearn.compose import make_column_selector
# from sklearn.ensemble import StackingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score
from sklearn.model_selection import train_test_split

import My_Transformers
from sklearn.preprocessing import StandardScaler
# Global var
TRAIN_ROW_COUNT = 8234
from My_Transformers import *
from Word2Vec_Transformer import Text_cleaner_transformer, STOPWORDS
import warnings
warnings.filterwarnings ("ignore")

# Global vars
VERSION = 9
trainDF, testDF, evalDF = None, None, None
sclf_1_1, sclf_1_2, sclf_1_3, sclf_2_1 = None, None, None, None
lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2 = None, None, None, None, None, None, None, None, None, None
X_train, X_test, X_eval, tfidf_slice, bert_meta_slice, bert_meta_ica_slice, tfidf_pca_nmf_slice = None, None, None, None, None, None, None
TRAIN_SAMPLE_WEIGHT = None
CACHEDIR = mkdtemp()
BEST_CLF = None

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-10-03 15:35:44,773 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-10-03 15:35:44,774 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):

    if VERSION != '':

        file, ext = os.path.splitext (file)
        file += "_v" + str (VERSION) + ext
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):

    if VERSION != '':

        file, ext = os.path.splitext(file)
        file += "_v" + str(VERSION) + ext
    return pickle.load (open (file, 'rb'))

In [3]:
class Discrete2OneHot_FeatureTransformer (BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        self.LB_1 = None
        self.LB_2 = None
        return

    def fit (self, X, y=None, **fit_params):
        
        print ('Discrete2OneHot_FeatureTransformer: fit(): X.shape =', X.shape)
        self.LB_1 = LabelBinarizer ()
        X['keyword'] = X.keyword.astype (str)
        self.LB_1.fit (list (X['keyword']))
        # self.LB_2 = LabelBinarizer ()
        # self.LB_2.fit (list (pd.isnull (X['location'])))
        return self

    def transform (self, X, y=None, **fit_params):
        
        X['keyword'] = X.keyword.astype (str)
        A = self.LB_1.transform (list (X['keyword']))
        # B = self.LB_2.transform (list (pd.isnull (X['location']))).reshape (-1, 1)
        # X = np.hstack ((A, B))
        X = A
        print ('Discrete2OneHot_FeatureTransformer: transform(): X.shape =', X.shape)
        return X

# FV Trainers

In [4]:
# Global vars

tfidf_ft = Pipeline ([ 
    
    ('tfidf_text_meta_keyword', ColumnTransformer ([ 
    
        ('tfidf_text', Pipeline ([

            ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 8), max_df=0.90, min_df=5, token_pattern='(\S+)')),
            ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
            ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
            ('standardization1',        StandardScaler (with_mean=False)),
            ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
        ], memory=CACHEDIR), 'text_cleaned'),
        # ('tfidf_location',  TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)'), 'location'),
        ('meta_features', 'passthrough', ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length', 'char_count', 'punctuation_count', 'hashtag_count', 'mention_count']),
        ('keyword',        Discrete2OneHot_FeatureTransformer (), ['keyword']),
    ])), 
    ('cosineSim',         Cosine_sim_ratios_Features ()),
    ('standardization2',  StandardScaler (with_mean=False)),
    ('k_best',            SelectKBest_feature_selector (n_components=[0.8, 0.9, 0.95], score_func=[mutual_info_classif]))
], memory=CACHEDIR)

tfidf_pca_nmf_ft = Pipeline ([ 

    ('tfidf_ft', tfidf_ft),
    ('pca_nmf',  PCA_NMF_TrainTest_FeatureTransformer (isSparseOut=True)),
    ('k_best',   SelectKBest_feature_selector (n_components=[0.8, 0.9, 0.95], score_func=[mutual_info_classif]))
], memory=CACHEDIR)

bert_ft = ColumnTransformer ([
    
    ('bert', 'passthrough', make_column_selector (r'bert')),
])

bert_meta_ft = ColumnTransformer ([
    
    ('bert',          'passthrough', make_column_selector (r'bert')),
    # ('keyword',        Discrete2OneHot_FeatureTransformer (), ['keyword']),
    ('meta_features', 'passthrough', ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length', 'char_count', 'punctuation_count', 'hashtag_count', 'mention_count'])
])

bert_meta_keyword_ica_ft = Pipeline ([
    
    ('bert_keyword_meta', ColumnTransformer ([ 
        
        ('bert',          'passthrough', make_column_selector (r'bert')),
        ('keyword',        Discrete2OneHot_FeatureTransformer (), ['keyword']),
        ('meta_features', 'passthrough', ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length', 'char_count', 'punctuation_count', 'hashtag_count', 'mention_count'])
    ])),
    ('cosineSim',  Cosine_sim_ratios_Features ()),
    ('kpca_ica',  FeatureUnion([
        
        ('kpca',   KPCA_TrainTest_FeatureTransformer (kernel=['cosine'], isSparseOut=True)),
        ('ica',    ICA_TrainTest_FeatureTransformer (isSparseOut=True))
    ])),
    ('k_best',     SelectKBest_feature_selector (n_components=[0.8, 0.9, 0.95], score_func=[mutual_info_classif]))
])

In [5]:
def fit_transform_fv (df):
    
    global tfidf_ft, bert_meta_ft, bert_meta_keyword_ica_ft, tfidf_pca_nmf_ft
    global tfidf_slice, bert_meta_slice, bert_meta_ica_slice, tfidf_pca_nmf_slice
    
    print ('fit_transform_fv () Begin with tfidf_ft.fit_transform ():')
    X_tfidf         = tfidf_ft.fit_transform (df, df['target_relabeled'])
    print ('bert_meta_ft.fit_transform ():')
    X_bert_meta     = bert_meta_ft.fit_transform (df, df['target_relabeled'])
    print ('bert_meta_keyword_ica_ft.fit_transform ():')
    X_bert_meta_ica = bert_meta_keyword_ica_ft.fit_transform (df, df['target_relabeled'])
    print ('tfidf_pca_nmf_ft.fit_transform ():')
    X_tfidf_pca_nmf = tfidf_pca_nmf_ft.fit_transform (df, df['target_relabeled'])
    
    X = scipy.sparse.hstack ([X_tfidf, X_bert_meta, X_bert_meta_ica, X_tfidf_pca_nmf])
    X = X.tocsr ()    
    tfidf_slice         = slice (0, X_tfidf.shape[1])
    bert_meta_slice     = slice (X_tfidf.shape[1],  X_tfidf.shape[1] + X_bert_meta.shape[1])
    bert_meta_ica_slice = slice (X_tfidf.shape[1] + X_bert_meta.shape[1], X_tfidf.shape[1] + X_bert_meta.shape[1] + X_bert_meta_ica.shape[1])
    tfidf_pca_nmf_slice = slice (X_tfidf.shape[1] + X_bert_meta.shape[1] + X_bert_meta_ica.shape[1], X_tfidf.shape[1] + X_bert_meta.shape[1] + X_bert_meta_ica.shape[1] + X_tfidf_pca_nmf.shape[1])
    print ('fit_transform_fv () Done.')
    return X

In [6]:
def transform_fv (df):
    
    global tfidf_ft, bert_meta_ft, bert_meta_keyword_ica_ft, tfidf_pca_nmf_ft
    global tfidf_slice, bert_meta_slice, bert_meta_ica_slice, tfidf_pca_nmf_slice
    
    X_tfidf         = tfidf_ft.transform (df)
    X_bert_meta     = bert_meta_ft.transform (df)   
    X_bert_meta_ica = bert_meta_keyword_ica_ft.transform (df)
    X_tfidf_pca_nmf = tfidf_pca_nmf_ft.transform (df)
    
    X = scipy.sparse.hstack ([X_tfidf, X_bert_meta, X_bert_meta_ica, X_tfidf_pca_nmf])
    X = X.tocsr()
    return X

# L-0 Classifiers

In [7]:
def create_fit_L0_clf (X, y):
    
    global lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2
    global tfidf_slice, bert_meta_slice, bert_meta_ica_slice, tfidf_pca_nmf_slice
    
    # Create
    lsvc_1 = Pipeline ([ 
        ('col_select', ColumnTransformer ([('col_select', 'passthrough', tfidf_slice) ])),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc', 'n_estimators': 20, 'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }))
    ])
    lsvc_1.name = 'lsvc_1'

    lsvc_2 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc', 'n_estimators': 20,  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }))
    ])
    lsvc_2.name = 'lsvc_2'
    
    lsvc_3 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_ica_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc', 'n_estimators': 20,  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }))
    ])
    lsvc_3.name = 'lsvc_3'
    
    lsvc_4 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', tfidf_pca_nmf_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc', 'n_estimators': 20,  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }))
    ])
    lsvc_4.name = 'lsvc_4'


    catb_1 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', tfidf_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ])
    catb_1.name = 'catb_1'

    catb_2 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ])
    catb_2.name = 'catb_2'
    
    catb_3 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_ica_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ])
    catb_3.name = 'catb_3'
    
    catb_4 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', tfidf_pca_nmf_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ])
    catb_4.name = 'catb_4'
    
    
    fcnn_1 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'FCNN', 'max_epochs': 100, 'n_estimators': 20}))
    ])
    fcnn_1.name = 'fcnn_1'
    
    fcnn_2 = Pipeline ([ 
         ('col_select', ColumnTransformer ([('col_select', 'passthrough', bert_meta_ica_slice) ])),
         ('clf',        Best_clf_cv_transformer ({ 'name': 'FCNN', 'max_epochs': 100, 'n_estimators': 20}))
    ])
    fcnn_2.name = 'fcnn_2'
    
    # fit
    lsvc_1.fit (X, y) 
    lsvc_2.fit (X, y) 
    lsvc_3.fit (X, y) 
    lsvc_4.fit (X, y) 
    catb_1.fit (X, y)
    catb_2.fit (X, y) 
    catb_3.fit (X, y)
    catb_4.fit (X, y)
    fcnn_1.fit (X, y)
    fcnn_2.fit (X, y)
    return

# Stacking Classifiers

In [8]:
def create_fit_stack_clf (X, y):
    
    global sclf_1_1, sclf_1_2, sclf_1_3, sclf_2_1
    global lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2
    
    # Level 1 create
    sclf_1_1 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    sclf_1_1.name = 'sclf_1_1'

    sclf_1_2 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'RF',    'params': {'class_weight': 'balanced', 'n_estimators': 3, 'max_features': None} }))
    sclf_1_2.name = 'sclf_1_2'

    sclf_1_3 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, lsvc_4, catb_1, catb_2, catb_3, catb_4, fcnn_1, fcnn_2], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False }))
    sclf_1_3.name = 'sclf_1_3'
    
    # fit
    sclf_1_1.fit (X, y)
    sclf_1_2.fit (X, y)
    sclf_1_3.fit (X, y)

    # Level 2 create
    sclf_2_1 = StackingClassifier (classifiers=[sclf_1_1, sclf_1_2, sclf_1_3], fit_base_estimators=False, 
                                   use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    sclf_2_1.name = 'sclf_2_1'
    
    # fit
    sclf_2_1.fit (X, y)
    return    

# sample weights - don't know how to use this ? How to deal with unbalanced classes ?

In [9]:
def get_sample_weights (df):
    
    sample_weights = []
    y = df[pd.isnull (df['target_relabeled']) == False]['target_relabeled']
    a = 1.0/ np.sum (y)                # class weight for label 1
    b = 1.0/ (len (y) - np.sum (y))    # class weight for label 0
    w1 = a/(a+b)
    w0 = b/(a+b)
    for i in y:
        if i:
            sample_weights.append (w1)
        else:
            sample_weights.append (w0)
    return sample_weights, b, a

In [10]:
def init ():
    
    global trainDF, evalDF, TRAIN_ROW_COUNT
    
    train_file = "Data/all.tsv"
    rawDF = pd.read_csv (train_file, sep='\t')  #, dtype={'id': np.int16, 'target': np.int8})
    # rawDF = rawDF[0:30]               # Comment this out ********************************
    rawDF['text'] = rawDF.text.astype (str)
    rawDF['keyword'] = rawDF.keyword.astype (str)
    rawDF['location'] = rawDF.location.astype (str)
    rawDF['text_cleaned'] = rawDF.text_cleaned.astype (str)
    # rawDF['target_relabeled'] = rawDF.target_relabeled.astype (int)
    
    labeledDF = rawDF[:TRAIN_ROW_COUNT]
    trainDF, evalDF = train_test_split (labeledDF, test_size=0.33, random_state=42)
    evalDF['target_relabeled'] = evalDF.target_relabeled.astype (int)
    trainDF['target_relabeled'] = trainDF.target_relabeled.astype (int)
    trainDF_temp = pd.concat ((trainDF, rawDF[TRAIN_ROW_COUNT:]), axis=0)
    TRAIN_ROW_COUNT = trainDF.shape[0]
    trainDF = trainDF_temp
    
    print ('rawDF.shape =', rawDF.shape, 'trainDF.shape =', trainDF.shape, 'evalDF.shape =', evalDF.shape)
    rawDF, labeledDF, trainDF_temp = None, None, None
    return

In [11]:
def fit (clf=None, isFitOnOrigDF=False):
    
    global trainDF, X_train
    
    if isFitOnOrigDF:
        
        train_file = "Data/all.tsv"
        trainDF = pd.read_csv (train_file, sep='\t')    #, dtype={'id': np.int16, 'target': np.int8})
        trainDF['text'] = trainDF.text.astype (str)
        trainDF['keyword'] = trainDF.keyword.astype (str)
        trainDF['location'] = trainDF.location.astype (str)
        trainDF['text_cleaned'] = trainDF.text_cleaned.astype (str)
        # trainDF = trainDF[0:20]               # Comment this out ********************************
    X_train = fit_transform_fv (trainDF)    
    if clf is None:
        
        create_fit_L0_clf (X_train, trainDF['target_relabeled'])
        create_fit_stack_clf (X_train, trainDF['target_relabeled'])
    else:
        
        clf.fit (X_train, trainDF['target_relabeled'])
    return

In [12]:
def predict (clf, iterCount=0):
    
    global testDF, X_test
    
    print ("Starting Prediction by", clf.name)
    test_file = "Data/test.tsv"
    testDF = pd.read_csv (test_file, sep='\t')   #, dtype={'id': np.int16, 'target': np.int8})
    # testDF = testDF[0:10]              # Comment this out ********************************
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    testDF['location'] = testDF.location.astype (str)
    testDF['text_cleaned'] = testDF.text_cleaned.astype (str)
    Y_perfect = testDF['target'].astype (int)
    X_test = transform_fv (testDF)
    
    predictions      = np.round (clf.predict (X_test)).astype (int)
    testDF['target'] = predictions
    testDF['id']     = testDF.id.astype (int)
    testDF['target'] = testDF.target.astype (int)
    submitDF         = testDF[['id', 'target']]
    submitDF.to_csv ('submission_v'+str (VERSION) + clf.name + '.' + str (iterCount+1) + '.csv', index=False)
    f1_acc = f1_score (Y_perfect, predictions)
    print ('\n************** F1-Score =', f1_acc, "*************\n")
    print ("Done Prediction !")
    return

In [13]:
def get_best_evaluation ():
    
    global evalDF
    
    clfs = [lsvc_1, catb_1, lsvc_2, catb_2, lsvc_3, catb_3, lsvc_4, catb_4, fcnn_1, fcnn_2, sclf_1_1, sclf_1_2, sclf_1_3, sclf_2_1]
    accs = []
    X_eval = transform_fv (evalDF)
    for clf in clfs:
        
        print ('------------------------- evaluating:', clf.name, '-------------------------')
        predictions = clf.predict (X_eval)
        f1_acc = f1_score(evalDF['target_relabeled'], predictions)
        accs.append (f1_acc)
        print ('------------------------- '+clf.name+' f1_score =', f1_acc, '-------------------------')
    
    best_clf = clfs [accs.index (max (accs))]
    print ('Best clf =', best_clf.name)
    return best_clf

In [14]:
# init ()
# fit ()
# get_best_evaluation ()
# fit (isFitOnOrigDF=True)
# fit ()
# predict (sclf_2_1)

In [15]:
def main (threshold=0.9, maxIterCount=1, thresh_depreciation=0.01):
        
    global trainDF, X_train, BEST_CLF, TRAIN_ROW_COUNT, sclf_2_1

    
    init ()
    fit ()
    BEST_CLF = get_best_evaluation ()
    newTrainAddCount = 1
    iterCount = 0
    while newTrainAddCount > 0 and iterCount < maxIterCount:
        
        threshold = threshold - iterCount * thresh_depreciation
        if iterCount == 0:
            fit (isFitOnOrigDF=True)
        else:
            fit ()
        predict (sclf_2_1, iterCount)
        predict (BEST_CLF, iterCount)
        Pr = BEST_CLF.predict_proba (X_train)
        newTrainAddCount = 0
        for idx, row in trainDF.iterrows ():
            if not pd.isnull (row['target_relabeled']):
                continue
                
            # print ('considering idx =', idx)
            pr = Pr[idx]
            p  = 0
            if pr[0] > 0.5:
                pr = pr[0]
                p  = 0
            else:
                pr = pr[1]
                p  = 1
            if pr >= threshold:
                # add in the training set
                trainDF.at[idx, 'target_relabeled'] = p
                newTrainAddCount += 1
        # arrange df such that 1st rows with y=0/1 then rows with y=NaN
        arrangedDF1 = trainDF[pd.isnull (trainDF['target_relabeled']) == False]
        arrangedDF2 = trainDF[pd.isnull (trainDF['target_relabeled'])]
        trainDF = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        iterCount += 1
    trainDF.to_csv ('Data/all_'+str (VERSION) + BEST_CLF.name + '.' + str (iterCount+1)+'.csv', index=False)
    fit ()
    predict (sclf_2_1, iterCount)
    predict (BEST_CLF, iterCount)
    
    # CV for the BEST_CLF on the original train rows
    print ('------------------- CV for ' + BEST_CLF.name + ' on the original train rows -------------------')
    TRAIN_ROW_COUNT += evalDF.shape[0]
    trainDF = trainDF[:TRAIN_ROW_COUNT]
    cv_results = []
    cv_result = cross_validate (BEST_CLF, trainDF, cv=5)
    cv_score = np.mean (cv_result['test_score'])
    print ()
    print (BEST_CLF.name, cv_score)
    print (cv_result)
    rmtree (CACHEDIR)
    return

# ROC Curves

In [16]:
def adjusted_classes (y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    Will only work for binary classification problems.
    """
    return [1 if y >= t else 0 for y in y_scores]

In [17]:
def plot_precision_recall_vs_threshold (precisions, recalls, thresholds):
    """
    Modified from:
    Hands-On Machine learning with Scikit-Learn
    and TensorFlow; p.89
    """
    
    plt.figure(figsize=(8, 8))
    plt.title("Precision and Recall Scores as a function of the decision threshold")
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.ylabel("Score")
    plt.xlabel("Decision Threshold")
    plt.legend(loc='best')
    return

In [18]:
def plot_roc_curve (fpr, tpr, label=None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    
    plt.figure(figsize=(8,8))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.legend(loc='best')
    return

In [19]:
def roc_for_clf (clf):
    
    global TRAIN_ROW_COUNT, trainDF
    print ('-------------------------- ROC for', clf.name, '---------------------------')
    
    # init ()
    y_train_scores = clf.predict_proba (trainDF[:TRAIN_ROW_COUNT])[:, 1]
    y_train = list (trainDF[:TRAIN_ROW_COUNT]['target_relabeled'])
    
    # ROC curve : don't understand this ?
    fpr, tpr, auc_thresholds = roc_curve (y_train, y_train_scores)
    print ('AUC =', auc (fpr, tpr))         # AUC of ROC
    plot_roc_curve (fpr, tpr, 'recall_optimized')
    
    # plot Precisioin, Recall vs Threshold
    p, r, thresholds = precision_recall_curve (y_train, y_train_scores)
    plot_precision_recall_vs_threshold (p, r, thresholds)
    
    test_file = "Data/test.csv"
    testDF = pd.read_csv (test_file)
    # testDF = testDF[:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    testDF['location'] = testDF.location.astype (str)
    submitDF = testDF[['id', 'target']]
    y_test_scores = clf.predict_proba (testDF)[:, 1]
    for pr in thresholds:
        
        submitDF['target'] = adjusted_classes (y_test_scores, pr)
        submitDF['id']     = submitDF.id.astype (int)
        submitDF['target'] = submitDF.target.astype (int)
        print (pr, sum (list (submitDF['target'])))

In [20]:
def predict_threshold (clf, threshold_pr):
    
    test_file = "Data/test.tsv"
    testDF = pd.read_csv (test_file, sep='\t')
    # testDF = testDF[:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    testDF['location'] = testDF.location.astype (str)
    Y_perfect = testDF['target'].astype (int)
    submitDF = testDF[['id', 'target']]
    y_test_scores = clf.predict_proba (testDF)[:, 1]
    
    predictions        = adjusted_classes (y_test_scores, threshold_pr)
    submitDF['target'] = predictions
    submitDF['id']     = submitDF.id.astype (int)
    submitDF['target'] = submitDF.target.astype (int)
    submitDF.to_csv ('submission_v'+str (VERSION) + clf.name + '.roc'+ str (threshold_pr) + '.csv', index=False)
    f1_acc = f1_score (Y_perfect, predictions)
    print ('\n************** F1-Score =', f1_acc, "*************\n")

# Main

In [21]:
main (threshold=0.93, maxIterCount=1)
roc_for_clf (BEST_CLF)

rawDF.shape = (11497, 784) trainDF.shape = (8779, 784) evalDF.shape = (2718, 784)
fit_transform_fv () Begin with tfidf_ft.fit_transform ():
LinearSVC with L1-based feature selection for X.shape = (5527, 128291)
--------------------- Best parameter (CV score=0.738):
{'classifier__C': 1, 'feature_selection__estimator__tol': 0.0001}
New #features =  783
LinearSVC with L1-based feature selection for X.shape = (5527, 306936)
--------------------- Best parameter (CV score=0.721):
{'classifier__C': 0.001, 'feature_selection__estimator__tol': 0.0001}
New #features =  3282
Discrete2OneHot_FeatureTransformer: fit(): X.shape = (8779, 1)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (8779, 222)
Find K-best features based on Mutual-Info / Chi^2 for X.shape = (5527, 3515)
--------------------- Best parameter (CV score=0.961):
{'classifier__C': 10, 'selectKBest__k': 2812, 'selectKBest__score_func': <function mutual_info_classif at 0x000002A93E999950>}
bert_meta_ft.fit_transform ():
bert_

------------------------- sclf_2_1 f1_score = 0.4837791690381331 -------------------------
Best clf = lsvc_2
fit_transform_fv () Begin with tfidf_ft.fit_transform ():
LinearSVC with L1-based feature selection for X.shape = (8245, 161761)
--------------------- Best parameter (CV score=0.679):
{'classifier__C': 0.1, 'feature_selection__estimator__tol': 1e-06}
New #features =  1039
LinearSVC with L1-based feature selection for X.shape = (8245, 540280)
--------------------- Best parameter (CV score=0.637):
{'classifier__C': 0.001, 'feature_selection__estimator__tol': 0.0001}
New #features =  4524
Discrete2OneHot_FeatureTransformer: fit(): X.shape = (11497, 1)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11497, 222)
Find K-best features based on Mutual-Info / Chi^2 for X.shape = (8245, 4757)
--------------------- Best parameter (CV score=0.954):
{'classifier__C': 0.001, 'selectKBest__k': 4281, 'selectKBest__score_func': <function mutual_info_classif at 0x000002A93E999950>}
be

Done Fitting Catb
training Catb for X.shape = (8448, 777)
Done Fitting Catb
training Catb for X.shape = (8448, 1710)
Done Fitting Catb
training Catb for X.shape = (8448, 4252)
Done Fitting Catb
training FCNN for X.shape = (8448, 777)
FCNN : Best_clf_cv_transformer: starting CV = 5
FCNN : cv_score:   0.781
Done Fitting FCNN
training FCNN for X.shape = (8448, 1710)
FCNN : Best_clf_cv_transformer: starting CV = 5
FCNN : cv_score:   0.767
Done Fitting FCNN
training LSvc for X.shape = (8448, 20)
LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.989):
{'C': 0.01}
Done Fitting LSvc
training RF for X.shape = (8448, 20)
RF : Best_clf_cv_transformer: starting CV = 5
RF : cv_score:   0.986
Done Fitting RF
training Catb for X.shape = (8448, 20)
Done Fitting Catb
training LSvc for X.shape = (8448, 6)
LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.995):
{'C': 10}
Done Fitting LSvc
Starting Prediction by sclf_2_1
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 2

NameError: name 'clf' is not defined

In [22]:
predict (fcnn_1)

Starting Prediction by fcnn_1
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)

************** F1-Score = 0.7356760886172651 *************

Done Prediction !


In [23]:
predict (fcnn_2)

Starting Prediction by fcnn_2
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (3263, 222)

************** F1-Score = 0.7412378821774794 *************

Done Prediction !


In [ ]:
# predict_threshold (BEST_CLF, 0.9437731026789814)

# Additional

In [ ]:
def null_count ():
    
    global trainDF
    
    init ()
    # get total count of data including missing data
    total = trainDF.isnull ().sum ().sort_values (ascending=False)
    # print (total)

    # get percent of missing data relevant to all data
    percent = (trainDF.isnull ().sum ()/trainDF.isnull ().count ()).sort_values (ascending=False)
    # print (percent)
    
    missing_data = pd.concat ([total, percent], axis=1, keys=['total', 'percent'])
    return missing_data    

In [ ]:
def cleanTweet (tweet): 
            
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
            
    # Character entity references
    tweet = re.sub(r"&gt;", ">", tweet)
    tweet = re.sub(r"&lt;", "<", tweet)
    tweet = re.sub(r"&amp;", "&", tweet)
    
    # Typos, slang and informal abbreviations
    tweet = re.sub(r"w/e", "whatever", tweet)
    tweet = re.sub(r"w/", "with", tweet)
    tweet = re.sub(r"USAgov", "USA government", tweet)
    tweet = re.sub(r"recentlu", "recently", tweet)
    tweet = re.sub(r"Ph0tos", "Photos", tweet)
    tweet = re.sub(r"amirite", "am I right", tweet)
    tweet = re.sub(r"exp0sed", "exposed", tweet)
    tweet = re.sub(r"<3", "love", tweet)
    tweet = re.sub(r"amageddon", "armageddon", tweet)
    tweet = re.sub(r"Trfc", "Traffic", tweet)
    tweet = re.sub(r"8/5/2015", "2015-08-05", tweet)
    tweet = re.sub(r"WindStorm", "Wind Storm", tweet)
    tweet = re.sub(r"8/6/2015", "2015-08-06", tweet)
    tweet = re.sub(r"10:38PM", "10:38 PM", tweet)
    tweet = re.sub(r"10:30pm", "10:30 PM", tweet)
    tweet = re.sub(r"16yr", "16 year", tweet)
    tweet = re.sub(r"lmao", "laughing my ass off", tweet)   
    tweet = re.sub(r"TRAUMATISED", "traumatized", tweet)
    
    # Hashtags and usernames
    tweet = re.sub(r"IranDeal", "Iran Deal", tweet)
    tweet = re.sub(r"ArianaGrande", "Ariana Grande", tweet)
    tweet = re.sub(r"camilacabello97", "camila cabello", tweet) 
    tweet = re.sub(r"RondaRousey", "Ronda Rousey", tweet)     
    tweet = re.sub(r"MTVHottest", "MTV Hottest", tweet)
    tweet = re.sub(r"TrapMusic", "Trap Music", tweet)
    tweet = re.sub(r"ProphetMuhammad", "Prophet Muhammad", tweet)
    tweet = re.sub(r"PantherAttack", "Panther Attack", tweet)
    tweet = re.sub(r"StrategicPatience", "Strategic Patience", tweet)
    tweet = re.sub(r"socialnews", "social news", tweet)
    tweet = re.sub(r"NASAHurricane", "NASA Hurricane", tweet)
    tweet = re.sub(r"onlinecommunities", "online communities", tweet)
    tweet = re.sub(r"humanconsumption", "human consumption", tweet)
    tweet = re.sub(r"Typhoon-Devastated", "Typhoon Devastated", tweet)
    tweet = re.sub(r"Meat-Loving", "Meat Loving", tweet)
    tweet = re.sub(r"facialabuse", "facial abuse", tweet)
    tweet = re.sub(r"LakeCounty", "Lake County", tweet)
    tweet = re.sub(r"BeingAuthor", "Being Author", tweet)
    tweet = re.sub(r"withheavenly", "with heavenly", tweet)
    tweet = re.sub(r"thankU", "thank you", tweet)
    tweet = re.sub(r"iTunesMusic", "iTunes Music", tweet)
    tweet = re.sub(r"OffensiveContent", "Offensive Content", tweet)
    tweet = re.sub(r"WorstSummerJob", "Worst Summer Job", tweet)
    tweet = re.sub(r"HarryBeCareful", "Harry Be Careful", tweet)
    tweet = re.sub(r"NASASolarSystem", "NASA Solar System", tweet)
    tweet = re.sub(r"animalrescue", "animal rescue", tweet)
    tweet = re.sub(r"KurtSchlichter", "Kurt Schlichter", tweet)
    tweet = re.sub(r"aRmageddon", "armageddon", tweet)
    tweet = re.sub(r"Throwingknifes", "Throwing knives", tweet)
    tweet = re.sub(r"GodsLove", "God's Love", tweet)
    tweet = re.sub(r"bookboost", "book boost", tweet)
    tweet = re.sub(r"ibooklove", "I book love", tweet)
    tweet = re.sub(r"NestleIndia", "Nestle India", tweet)
    tweet = re.sub(r"realDonaldTrump", "Donald Trump", tweet)
    tweet = re.sub(r"DavidVonderhaar", "David Vonderhaar", tweet)
    tweet = re.sub(r"CecilTheLion", "Cecil The Lion", tweet)
    tweet = re.sub(r"weathernetwork", "weather network", tweet)
    tweet = re.sub(r"withBioterrorism&use", "with Bioterrorism & use", tweet)
    tweet = re.sub(r"Hostage&2", "Hostage & 2", tweet)
    tweet = re.sub(r"GOPDebate", "GOP Debate", tweet)
    tweet = re.sub(r"RickPerry", "Rick Perry", tweet)
    tweet = re.sub(r"frontpage", "front page", tweet)
    tweet = re.sub(r"NewsInTweets", "News In Tweets", tweet)
    tweet = re.sub(r"ViralSpell", "Viral Spell", tweet)
    tweet = re.sub(r"til_now", "until now", tweet)
    tweet = re.sub(r"volcanoinRussia", "volcano in Russia", tweet)
    tweet = re.sub(r"ZippedNews", "Zipped News", tweet)
    tweet = re.sub(r"MicheleBachman", "Michele Bachman", tweet)
    tweet = re.sub(r"53inch", "53 inch", tweet)
    tweet = re.sub(r"KerrickTrial", "Kerrick Trial", tweet)
    tweet = re.sub(r"abstorm", "Alberta Storm", tweet)
    tweet = re.sub(r"Beyhive", "Beyonce hive", tweet)
    tweet = re.sub(r"IDFire", "Idaho Fire", tweet)
    tweet = re.sub(r"DETECTADO", "Detected", tweet)
    tweet = re.sub(r"RockyFire", "Rocky Fire", tweet)
    tweet = re.sub(r"Listen/Buy", "Listen / Buy", tweet)
    tweet = re.sub(r"NickCannon", "Nick Cannon", tweet)
    tweet = re.sub(r"FaroeIslands", "Faroe Islands", tweet)
    tweet = re.sub(r"yycstorm", "Calgary Storm", tweet)
    tweet = re.sub(r"IDPs:", "Internally Displaced People :", tweet)
    tweet = re.sub(r"ArtistsUnited", "Artists United", tweet)
    tweet = re.sub(r"ClaytonBryant", "Clayton Bryant", tweet)
    tweet = re.sub(r"jimmyfallon", "jimmy fallon", tweet)
    tweet = re.sub(r"justinbieber", "justin bieber", tweet)  
    tweet = re.sub(r"UTC2015", "UTC 2015", tweet)
    tweet = re.sub(r"Time2015", "Time 2015", tweet)
    tweet = re.sub(r"djicemoon", "dj icemoon", tweet)
    tweet = re.sub(r"LivingSafely", "Living Safely", tweet)
    tweet = re.sub(r"FIFA16", "Fifa 2016", tweet)
    tweet = re.sub(r"thisiswhywecanthavenicethings", "this is why we cannot have nice things", tweet)
    tweet = re.sub(r"bbcnews", "bbc news", tweet)
    tweet = re.sub(r"UndergroundRailraod", "Underground Railraod", tweet)
    tweet = re.sub(r"c4news", "c4 news", tweet)
    tweet = re.sub(r"OBLITERATION", "obliteration", tweet)
    tweet = re.sub(r"MUDSLIDE", "mudslide", tweet)
    tweet = re.sub(r"NoSurrender", "No Surrender", tweet)
    tweet = re.sub(r"NotExplained", "Not Explained", tweet)
    tweet = re.sub(r"greatbritishbakeoff", "great british bake off", tweet)
    tweet = re.sub(r"LondonFire", "London Fire", tweet)
    tweet = re.sub(r"KOTAWeather", "KOTA Weather", tweet)
    tweet = re.sub(r"LuchaUnderground", "Lucha Underground", tweet)
    tweet = re.sub(r"KOIN6News", "KOIN 6 News", tweet)
    tweet = re.sub(r"LiveOnK2", "Live On K2", tweet)
    tweet = re.sub(r"9NewsGoldCoast", "9 News Gold Coast", tweet)
    tweet = re.sub(r"nikeplus", "nike plus", tweet)
    tweet = re.sub(r"david_cameron", "David Cameron", tweet)
    tweet = re.sub(r"peterjukes", "Peter Jukes", tweet)
    tweet = re.sub(r"JamesMelville", "James Melville", tweet)
    tweet = re.sub(r"megynkelly", "Megyn Kelly", tweet)
    tweet = re.sub(r"cnewslive", "C News Live", tweet)
    tweet = re.sub(r"JamaicaObserver", "Jamaica Observer", tweet)
    tweet = re.sub(r"TweetLikeItsSeptember11th2001", "Tweet like it is september 11th 2001", tweet)
    tweet = re.sub(r"cbplawyers", "cbp lawyers", tweet)
    tweet = re.sub(r"fewmoretweets", "few more tweets", tweet)
    tweet = re.sub(r"BlackLivesMatter", "Black Lives Matter", tweet)
    tweet = re.sub(r"cjoyner", "Chris Joyner", tweet)
    tweet = re.sub(r"ENGvAUS", "England vs Australia", tweet)
    tweet = re.sub(r"ScottWalker", "Scott Walker", tweet)
    tweet = re.sub(r"MikeParrActor", "Michael Parr", tweet)
    tweet = re.sub(r"4PlayThursdays", "Foreplay Thursdays", tweet)
    tweet = re.sub(r"TGF2015", "Tontitown Grape Festival", tweet)
    tweet = re.sub(r"realmandyrain", "Mandy Rain", tweet)
    tweet = re.sub(r"GraysonDolan", "Grayson Dolan", tweet)
    tweet = re.sub(r"ApolloBrown", "Apollo Brown", tweet)
    tweet = re.sub(r"saddlebrooke", "Saddlebrooke", tweet)
    tweet = re.sub(r"TontitownGrape", "Tontitown Grape", tweet)
    tweet = re.sub(r"AbbsWinston", "Abbs Winston", tweet)
    tweet = re.sub(r"ShaunKing", "Shaun King", tweet)
    tweet = re.sub(r"MeekMill", "Meek Mill", tweet)
    tweet = re.sub(r"TornadoGiveaway", "Tornado Giveaway", tweet)
    tweet = re.sub(r"GRupdates", "GR updates", tweet)
    tweet = re.sub(r"SouthDowns", "South Downs", tweet)
    tweet = re.sub(r"braininjury", "brain injury", tweet)
    tweet = re.sub(r"auspol", "Australian politics", tweet)
    tweet = re.sub(r"PlannedParenthood", "Planned Parenthood", tweet)
    tweet = re.sub(r"calgaryweather", "Calgary Weather", tweet)
    tweet = re.sub(r"weallheartonedirection", "we all heart one direction", tweet)
    tweet = re.sub(r"edsheeran", "Ed Sheeran", tweet)
    tweet = re.sub(r"TrueHeroes", "True Heroes", tweet)
    tweet = re.sub(r"S3XLEAK", "sex leak", tweet)
    tweet = re.sub(r"ComplexMag", "Complex Magazine", tweet)
    tweet = re.sub(r"TheAdvocateMag", "The Advocate Magazine", tweet)
    tweet = re.sub(r"CityofCalgary", "City of Calgary", tweet)
    tweet = re.sub(r"EbolaOutbreak", "Ebola Outbreak", tweet)
    tweet = re.sub(r"SummerFate", "Summer Fate", tweet)
    tweet = re.sub(r"RAmag", "Royal Academy Magazine", tweet)
    tweet = re.sub(r"offers2go", "offers to go", tweet)
    tweet = re.sub(r"foodscare", "food scare", tweet)
    tweet = re.sub(r"MNPDNashville", "Metropolitan Nashville Police Department", tweet)
    tweet = re.sub(r"TfLBusAlerts", "TfL Bus Alerts", tweet)
    tweet = re.sub(r"GamerGate", "Gamer Gate", tweet)
    tweet = re.sub(r"IHHen", "Humanitarian Relief", tweet)
    tweet = re.sub(r"spinningbot", "spinning bot", tweet)
    tweet = re.sub(r"ModiMinistry", "Modi Ministry", tweet)
    tweet = re.sub(r"TAXIWAYS", "taxi ways", tweet)
    tweet = re.sub(r"Calum5SOS", "Calum Hood", tweet)
    tweet = re.sub(r"po_st", "po.st", tweet)
    tweet = re.sub(r"scoopit", "scoop.it", tweet)
    tweet = re.sub(r"UltimaLucha", "Ultima Lucha", tweet)
    tweet = re.sub(r"JonathanFerrell", "Jonathan Ferrell", tweet)
    tweet = re.sub(r"aria_ahrary", "Aria Ahrary", tweet)
    tweet = re.sub(r"rapidcity", "Rapid City", tweet)
    tweet = re.sub(r"OutBid", "outbid", tweet)
    tweet = re.sub(r"lavenderpoetrycafe", "lavender poetry cafe", tweet)
    tweet = re.sub(r"EudryLantiqua", "Eudry Lantiqua", tweet)
    tweet = re.sub(r"15PM", "15 PM", tweet)
    tweet = re.sub(r"OriginalFunko", "Funko", tweet)
    tweet = re.sub(r"rightwaystan", "Richard Tan", tweet)
    tweet = re.sub(r"CindyNoonan", "Cindy Noonan", tweet)
    tweet = re.sub(r"RT_America", "RT America", tweet)
    tweet = re.sub(r"narendramodi", "Narendra Modi", tweet)
    tweet = re.sub(r"BakeOffFriends", "Bake Off Friends", tweet)
    tweet = re.sub(r"TeamHendrick", "Hendrick Motorsports", tweet)
    tweet = re.sub(r"alexbelloli", "Alex Belloli", tweet)
    tweet = re.sub(r"itsjustinstuart", "Justin Stuart", tweet)
    tweet = re.sub(r"gunsense", "gun sense", tweet)
    tweet = re.sub(r"DebateQuestionsWeWantToHear", "debate questions we want to hear", tweet)
    tweet = re.sub(r"RoyalCarribean", "Royal Carribean", tweet)
    tweet = re.sub(r"samanthaturne19", "Samantha Turner", tweet)
    tweet = re.sub(r"JonVoyage", "Jon Stewart", tweet)
    tweet = re.sub(r"renew911health", "renew 911 health", tweet)
    tweet = re.sub(r"SuryaRay", "Surya Ray", tweet)
    tweet = re.sub(r"pattonoswalt", "Patton Oswalt", tweet)
    tweet = re.sub(r"minhazmerchant", "Minhaz Merchant", tweet)
    tweet = re.sub(r"TLVFaces", "Israel Diaspora Coalition", tweet)
    tweet = re.sub(r"pmarca", "Marc Andreessen", tweet)
    tweet = re.sub(r"pdx911", "Portland Police", tweet)
    tweet = re.sub(r"jamaicaplain", "Jamaica Plain", tweet)
    tweet = re.sub(r"Japton", "Arkansas", tweet)
    tweet = re.sub(r"RouteComplex", "Route Complex", tweet)
    tweet = re.sub(r"INSubcontinent", "Indian Subcontinent", tweet)
    tweet = re.sub(r"NJTurnpike", "New Jersey Turnpike", tweet)
    tweet = re.sub(r"Politifiact", "PolitiFact", tweet)
    tweet = re.sub(r"Hiroshima70", "Hiroshima", tweet)
    tweet = re.sub(r"GMMBC", "Greater Mt Moriah Baptist Church", tweet)
    tweet = re.sub(r"versethe", "verse the", tweet)
    tweet = re.sub(r"TubeStrike", "Tube Strike", tweet)
    tweet = re.sub(r"MissionHills", "Mission Hills", tweet)
    tweet = re.sub(r"ProtectDenaliWolves", "Protect Denali Wolves", tweet)
    tweet = re.sub(r"NANKANA", "Nankana", tweet)
    tweet = re.sub(r"SAHIB", "Sahib", tweet)
    tweet = re.sub(r"PAKPATTAN", "Pakpattan", tweet)
    tweet = re.sub(r"Newz_Sacramento", "News Sacramento", tweet)
    tweet = re.sub(r"gofundme", "go fund me", tweet)
    tweet = re.sub(r"pmharper", "Stephen Harper", tweet)
    tweet = re.sub(r"IvanBerroa", "Ivan Berroa", tweet)
    tweet = re.sub(r"LosDelSonido", "Los Del Sonido", tweet)
    tweet = re.sub(r"bancodeseries", "banco de series", tweet)
    tweet = re.sub(r"timkaine", "Tim Kaine", tweet)
    tweet = re.sub(r"IdentityTheft", "Identity Theft", tweet)
    tweet = re.sub(r"AllLivesMatter", "All Lives Matter", tweet)
    tweet = re.sub(r"mishacollins", "Misha Collins", tweet)
    tweet = re.sub(r"BillNeelyNBC", "Bill Neely", tweet)
    tweet = re.sub(r"BeClearOnCancer", "be clear on cancer", tweet)
    tweet = re.sub(r"Kowing", "Knowing", tweet)
    tweet = re.sub(r"ScreamQueens", "Scream Queens", tweet)
    tweet = re.sub(r"AskCharley", "Ask Charley", tweet)
    tweet = re.sub(r"BlizzHeroes", "Heroes of the Storm", tweet)
    tweet = re.sub(r"BradleyBrad47", "Bradley Brad", tweet)
    tweet = re.sub(r"HannaPH", "Typhoon Hanna", tweet)
    tweet = re.sub(r"meinlcymbals", "MEINL Cymbals", tweet)
    tweet = re.sub(r"Ptbo", "Peterborough", tweet)
    tweet = re.sub(r"cnnbrk", "CNN Breaking News", tweet)
    tweet = re.sub(r"IndianNews", "Indian News", tweet)
    tweet = re.sub(r"savebees", "save bees", tweet)
    tweet = re.sub(r"GreenHarvard", "Green Harvard", tweet)
    tweet = re.sub(r"StandwithPP", "Stand with planned parenthood", tweet)
    tweet = re.sub(r"hermancranston", "Herman Cranston", tweet)
    tweet = re.sub(r"WMUR9", "WMUR-TV", tweet)
    tweet = re.sub(r"RockBottomRadFM", "Rock Bottom Radio", tweet)
    tweet = re.sub(r"ameenshaikh3", "Ameen Shaikh", tweet)
    tweet = re.sub(r"ProSyn", "Project Syndicate", tweet)
    tweet = re.sub(r"Daesh", "ISIS", tweet)
    tweet = re.sub(r"s2g", "swear to god", tweet)
    tweet = re.sub(r"listenlive", "listen live", tweet)
    tweet = re.sub(r"CDCgov", "Centers for Disease Control and Prevention", tweet)
    tweet = re.sub(r"FoxNew", "Fox News", tweet)
    tweet = re.sub(r"CBSBigBrother", "Big Brother", tweet)
    tweet = re.sub(r"JulieDiCaro", "Julie DiCaro", tweet)
    tweet = re.sub(r"theadvocatemag", "The Advocate Magazine", tweet)
    tweet = re.sub(r"RohnertParkDPS", "Rohnert Park Police Department", tweet)
    tweet = re.sub(r"THISIZBWRIGHT", "Bonnie Wright", tweet)
    tweet = re.sub(r"Popularmmos", "Popular MMOs", tweet)
    tweet = re.sub(r"WildHorses", "Wild Horses", tweet)
    tweet = re.sub(r"FantasticFour", "Fantastic Four", tweet)
    tweet = re.sub(r"HORNDALE", "Horndale", tweet)
    tweet = re.sub(r"PINER", "Piner", tweet)
    tweet = re.sub(r"BathAndNorthEastSomerset", "Bath and North East Somerset", tweet)
    tweet = re.sub(r"thatswhatfriendsarefor", "that is what friends are for", tweet)
    tweet = re.sub(r"residualincome", "residual income", tweet)
    tweet = re.sub(r"YahooNewsDigest", "Yahoo News Digest", tweet)
    tweet = re.sub(r"MalaysiaAirlines", "Malaysia Airlines", tweet)
    tweet = re.sub(r"AmazonDeals", "Amazon Deals", tweet)
    tweet = re.sub(r"MissCharleyWebb", "Charley Webb", tweet)
    tweet = re.sub(r"shoalstraffic", "shoals traffic", tweet)
    tweet = re.sub(r"GeorgeFoster72", "George Foster", tweet)
    tweet = re.sub(r"pop2015", "pop 2015", tweet)
    tweet = re.sub(r"_PokemonCards_", "Pokemon Cards", tweet)
    tweet = re.sub(r"DianneG", "Dianne Gallagher", tweet)
    tweet = re.sub(r"KashmirConflict", "Kashmir Conflict", tweet)
    tweet = re.sub(r"BritishBakeOff", "British Bake Off", tweet)
    tweet = re.sub(r"FreeKashmir", "Free Kashmir", tweet)
    tweet = re.sub(r"mattmosley", "Matt Mosley", tweet)
    tweet = re.sub(r"BishopFred", "Bishop Fred", tweet)
    tweet = re.sub(r"EndConflict", "End Conflict", tweet)
    tweet = re.sub(r"EndOccupation", "End Occupation", tweet)
    tweet = re.sub(r"UNHEALED", "unhealed", tweet)
    tweet = re.sub(r"CharlesDagnall", "Charles Dagnall", tweet)
    tweet = re.sub(r"Latestnews", "Latest news", tweet)
    tweet = re.sub(r"KindleCountdown", "Kindle Countdown", tweet)
    tweet = re.sub(r"NoMoreHandouts", "No More Handouts", tweet)
    tweet = re.sub(r"datingtips", "dating tips", tweet)
    tweet = re.sub(r"charlesadler", "Charles Adler", tweet)
    tweet = re.sub(r"twia", "Texas Windstorm Insurance Association", tweet)
    tweet = re.sub(r"txlege", "Texas Legislature", tweet)
    tweet = re.sub(r"WindstormInsurer", "Windstorm Insurer", tweet)
    tweet = re.sub(r"Newss", "News", tweet)
    tweet = re.sub(r"hempoil", "hemp oil", tweet)
    tweet = re.sub(r"CommoditiesAre", "Commodities are", tweet)
    tweet = re.sub(r"tubestrike", "tube strike", tweet)
    tweet = re.sub(r"JoeNBC", "Joe Scarborough", tweet)
    tweet = re.sub(r"LiteraryCakes", "Literary Cakes", tweet)
    tweet = re.sub(r"TI5", "The International 5", tweet)
    tweet = re.sub(r"thehill", "the hill", tweet)
    tweet = re.sub(r"3others", "3 others", tweet)
    tweet = re.sub(r"stighefootball", "Sam Tighe", tweet)
    tweet = re.sub(r"whatstheimportantvideo", "what is the important video", tweet)
    tweet = re.sub(r"ClaudioMeloni", "Claudio Meloni", tweet)
    tweet = re.sub(r"DukeSkywalker", "Duke Skywalker", tweet)
    tweet = re.sub(r"carsonmwr", "Fort Carson", tweet)
    tweet = re.sub(r"offdishduty", "off dish duty", tweet)
    tweet = re.sub(r"andword", "and word", tweet)
    tweet = re.sub(r"rhodeisland", "Rhode Island", tweet)
    tweet = re.sub(r"easternoregon", "Eastern Oregon", tweet)
    tweet = re.sub(r"WAwildfire", "Washington Wildfire", tweet)
    tweet = re.sub(r"fingerrockfire", "Finger Rock Fire", tweet)
    tweet = re.sub(r"57am", "57 am", tweet)
    tweet = re.sub(r"fingerrockfire", "Finger Rock Fire", tweet)
    tweet = re.sub(r"JacobHoggard", "Jacob Hoggard", tweet)
    tweet = re.sub(r"newnewnew", "new new new", tweet)
    tweet = re.sub(r"under50", "under 50", tweet)
    tweet = re.sub(r"getitbeforeitsgone", "get it before it is gone", tweet)
    tweet = re.sub(r"freshoutofthebox", "fresh out of the box", tweet)
    tweet = re.sub(r"amwriting", "am writing", tweet)
    tweet = re.sub(r"Bokoharm", "Boko Haram", tweet)
    tweet = re.sub(r"Nowlike", "Now like", tweet)
    tweet = re.sub(r"seasonfrom", "season from", tweet)
    tweet = re.sub(r"epicente", "epicenter", tweet)
    tweet = re.sub(r"epicenterr", "epicenter", tweet)
    tweet = re.sub(r"sicklife", "sick life", tweet)
    tweet = re.sub(r"yycweather", "Calgary Weather", tweet)
    tweet = re.sub(r"calgarysun", "Calgary Sun", tweet)
    tweet = re.sub(r"approachng", "approaching", tweet)
    tweet = re.sub(r"evng", "evening", tweet)
    tweet = re.sub(r"Sumthng", "something", tweet)
    tweet = re.sub(r"EllenPompeo", "Ellen Pompeo", tweet)
    tweet = re.sub(r"shondarhimes", "Shonda Rhimes", tweet)
    tweet = re.sub(r"ABCNetwork", "ABC Network", tweet)
    tweet = re.sub(r"SushmaSwaraj", "Sushma Swaraj", tweet)
    tweet = re.sub(r"pray4japan", "Pray for Japan", tweet)
    tweet = re.sub(r"hope4japan", "Hope for Japan", tweet)
    tweet = re.sub(r"Illusionimagess", "Illusion images", tweet)
    tweet = re.sub(r"SummerUnderTheStars", "Summer Under The Stars", tweet)
    tweet = re.sub(r"ShallWeDance", "Shall We Dance", tweet)
    tweet = re.sub(r"TCMParty", "TCM Party", tweet)
    tweet = re.sub(r"marijuananews", "marijuana news", tweet)
    tweet = re.sub(r"onbeingwithKristaTippett", "on being with Krista Tippett", tweet)
    tweet = re.sub(r"Beingtweets", "Being tweets", tweet)
    tweet = re.sub(r"newauthors", "new authors", tweet)
    tweet = re.sub(r"remedyyyy", "remedy", tweet)
    tweet = re.sub(r"44PM", "44 PM", tweet)
    tweet = re.sub(r"HeadlinesApp", "Headlines App", tweet)
    tweet = re.sub(r"40PM", "40 PM", tweet)
    tweet = re.sub(r"myswc", "Severe Weather Center", tweet)
    tweet = re.sub(r"ithats", "that is", tweet)
    tweet = re.sub(r"icouldsitinthismomentforever", "I could sit in this moment forever", tweet)
    tweet = re.sub(r"FatLoss", "Fat Loss", tweet)
    tweet = re.sub(r"02PM", "02 PM", tweet)
    tweet = re.sub(r"MetroFmTalk", "Metro Fm Talk", tweet)
    tweet = re.sub(r"Bstrd", "bastard", tweet)
    tweet = re.sub(r"bldy", "bloody", tweet)
    tweet = re.sub(r"MetrofmTalk", "Metro Fm Talk", tweet)
    tweet = re.sub(r"terrorismturn", "terrorism turn", tweet)
    tweet = re.sub(r"BBCNewsAsia", "BBC News Asia", tweet)
    tweet = re.sub(r"BehindTheScenes", "Behind The Scenes", tweet)
    tweet = re.sub(r"GeorgeTakei", "George Takei", tweet)
    tweet = re.sub(r"WomensWeeklyMag", "Womens Weekly Magazine", tweet)
    tweet = re.sub(r"SurvivorsGuidetoEarth", "Survivors Guide to Earth", tweet)
    tweet = re.sub(r"incubusband", "incubus band", tweet)
    tweet = re.sub(r"Babypicturethis", "Baby picture this", tweet)
    tweet = re.sub(r"BombEffects", "Bomb Effects", tweet)
    tweet = re.sub(r"win10", "Windows 10", tweet)
    tweet = re.sub(r"idkidk", "I do not know I do not know", tweet)
    tweet = re.sub(r"TheWalkingDead", "The Walking Dead", tweet)
    tweet = re.sub(r"amyschumer", "Amy Schumer", tweet)
    tweet = re.sub(r"crewlist", "crew list", tweet)
    tweet = re.sub(r"Erdogans", "Erdogan", tweet)
    tweet = re.sub(r"BBCLive", "BBC Live", tweet)
    tweet = re.sub(r"TonyAbbottMHR", "Tony Abbott", tweet)
    tweet = re.sub(r"paulmyerscough", "Paul Myerscough", tweet)
    tweet = re.sub(r"georgegallagher", "George Gallagher", tweet)
    tweet = re.sub(r"JimmieJohnson", "Jimmie Johnson", tweet)
    tweet = re.sub(r"pctool", "pc tool", tweet)
    tweet = re.sub(r"DoingHashtagsRight", "Doing Hashtags Right", tweet)
    tweet = re.sub(r"ThrowbackThursday", "Throwback Thursday", tweet)
    tweet = re.sub(r"SnowBackSunday", "Snowback Sunday", tweet)
    tweet = re.sub(r"LakeEffect", "Lake Effect", tweet)
    tweet = re.sub(r"RTphotographyUK", "Richard Thomas Photography UK", tweet)
    tweet = re.sub(r"BigBang_CBS", "Big Bang CBS", tweet)
    tweet = re.sub(r"writerslife", "writers life", tweet)
    tweet = re.sub(r"NaturalBirth", "Natural Birth", tweet)
    tweet = re.sub(r"UnusualWords", "Unusual Words", tweet)
    tweet = re.sub(r"wizkhalifa", "Wiz Khalifa", tweet)
    tweet = re.sub(r"acreativedc", "a creative DC", tweet)
    tweet = re.sub(r"vscodc", "vsco DC", tweet)
    tweet = re.sub(r"VSCOcam", "vsco camera", tweet)
    tweet = re.sub(r"TheBEACHDC", "The beach DC", tweet)
    tweet = re.sub(r"buildingmuseum", "building museum", tweet)
    tweet = re.sub(r"WorldOil", "World Oil", tweet)
    tweet = re.sub(r"redwedding", "red wedding", tweet)
    tweet = re.sub(r"AmazingRaceCanada", "Amazing Race Canada", tweet)
    tweet = re.sub(r"WakeUpAmerica", "Wake Up America", tweet)
    tweet = re.sub(r"\\Allahuakbar\\", "Allahu Akbar", tweet)
    tweet = re.sub(r"bleased", "blessed", tweet)
    tweet = re.sub(r"nigeriantribune", "Nigerian Tribune", tweet)
    tweet = re.sub(r"HIDEO_KOJIMA_EN", "Hideo Kojima", tweet)
    tweet = re.sub(r"FusionFestival", "Fusion Festival", tweet)
    tweet = re.sub(r"50Mixed", "50 Mixed", tweet)
    tweet = re.sub(r"NoAgenda", "No Agenda", tweet)
    tweet = re.sub(r"WhiteGenocide", "White Genocide", tweet)
    tweet = re.sub(r"dirtylying", "dirty lying", tweet)
    tweet = re.sub(r"SyrianRefugees", "Syrian Refugees", tweet)
    tweet = re.sub(r"changetheworld", "change the world", tweet)
    tweet = re.sub(r"Ebolacase", "Ebola case", tweet)
    tweet = re.sub(r"mcgtech", "mcg technologies", tweet)
    tweet = re.sub(r"withweapons", "with weapons", tweet)
    tweet = re.sub(r"advancedwarfare", "advanced warfare", tweet)
    tweet = re.sub(r"letsFootball", "let us Football", tweet)
    tweet = re.sub(r"LateNiteMix", "late night mix", tweet)
    tweet = re.sub(r"PhilCollinsFeed", "Phil Collins", tweet)
    tweet = re.sub(r"RudyHavenstein", "Rudy Havenstein", tweet)
    tweet = re.sub(r"22PM", "22 PM", tweet)
    tweet = re.sub(r"54am", "54 AM", tweet)
    tweet = re.sub(r"38am", "38 AM", tweet)
    tweet = re.sub(r"OldFolkExplainStuff", "Old Folk Explain Stuff", tweet)
    tweet = re.sub(r"BlacklivesMatter", "Black Lives Matter", tweet)
    tweet = re.sub(r"InsaneLimits", "Insane Limits", tweet)
    tweet = re.sub(r"youcantsitwithus", "you cannot sit with us", tweet)
    tweet = re.sub(r"2k15", "2015", tweet)
    tweet = re.sub(r"TheIran", "Iran", tweet)
    tweet = re.sub(r"JimmyFallon", "Jimmy Fallon", tweet)
    tweet = re.sub(r"AlbertBrooks", "Albert Brooks", tweet)
    tweet = re.sub(r"defense_news", "defense news", tweet)
    tweet = re.sub(r"nuclearrcSA", "Nuclear Risk Control Self Assessment", tweet)
    tweet = re.sub(r"Auspol", "Australia Politics", tweet)
    tweet = re.sub(r"NuclearPower", "Nuclear Power", tweet)
    tweet = re.sub(r"WhiteTerrorism", "White Terrorism", tweet)
    tweet = re.sub(r"truthfrequencyradio", "Truth Frequency Radio", tweet)
    tweet = re.sub(r"ErasureIsNotEquality", "Erasure is not equality", tweet)
    tweet = re.sub(r"ProBonoNews", "Pro Bono News", tweet)
    tweet = re.sub(r"JakartaPost", "Jakarta Post", tweet)
    tweet = re.sub(r"toopainful", "too painful", tweet)
    tweet = re.sub(r"melindahaunton", "Melinda Haunton", tweet)
    tweet = re.sub(r"NoNukes", "No Nukes", tweet)
    tweet = re.sub(r"curryspcworld", "Currys PC World", tweet)
    tweet = re.sub(r"ineedcake", "I need cake", tweet)
    tweet = re.sub(r"blackforestgateau", "black forest gateau", tweet)
    tweet = re.sub(r"BBCOne", "BBC One", tweet)
    tweet = re.sub(r"AlexxPage", "Alex Page", tweet)
    tweet = re.sub(r"jonathanserrie", "Jonathan Serrie", tweet)
    tweet = re.sub(r"SocialJerkBlog", "Social Jerk Blog", tweet)
    tweet = re.sub(r"ChelseaVPeretti", "Chelsea Peretti", tweet)
    tweet = re.sub(r"irongiant", "iron giant", tweet)
    tweet = re.sub(r"RonFunches", "Ron Funches", tweet)
    tweet = re.sub(r"TimCook", "Tim Cook", tweet)
    tweet = re.sub(r"sebastianstanisaliveandwell", "Sebastian Stan is alive and well", tweet)
    tweet = re.sub(r"Madsummer", "Mad summer", tweet)
    tweet = re.sub(r"NowYouKnow", "Now you know", tweet)
    tweet = re.sub(r"concertphotography", "concert photography", tweet)
    tweet = re.sub(r"TomLandry", "Tom Landry", tweet)
    tweet = re.sub(r"showgirldayoff", "show girl day off", tweet)
    tweet = re.sub(r"Yougslavia", "Yugoslavia", tweet)
    tweet = re.sub(r"QuantumDataInformatics", "Quantum Data Informatics", tweet)
    tweet = re.sub(r"FromTheDesk", "From The Desk", tweet)
    tweet = re.sub(r"TheaterTrial", "Theater Trial", tweet)
    tweet = re.sub(r"CatoInstitute", "Cato Institute", tweet)
    tweet = re.sub(r"EmekaGift", "Emeka Gift", tweet)
    tweet = re.sub(r"LetsBe_Rational", "Let us be rational", tweet)
    tweet = re.sub(r"Cynicalreality", "Cynical reality", tweet)
    tweet = re.sub(r"FredOlsenCruise", "Fred Olsen Cruise", tweet)
    tweet = re.sub(r"NotSorry", "not sorry", tweet)
    tweet = re.sub(r"UseYourWords", "use your words", tweet)
    tweet = re.sub(r"WordoftheDay", "word of the day", tweet)
    tweet = re.sub(r"Dictionarycom", "Dictionary.com", tweet)
    tweet = re.sub(r"TheBrooklynLife", "The Brooklyn Life", tweet)
    tweet = re.sub(r"jokethey", "joke they", tweet)
    tweet = re.sub(r"nflweek1picks", "NFL week 1 picks", tweet)
    tweet = re.sub(r"uiseful", "useful", tweet)
    tweet = re.sub(r"JusticeDotOrg", "The American Association for Justice", tweet)
    tweet = re.sub(r"autoaccidents", "auto accidents", tweet)
    tweet = re.sub(r"SteveGursten", "Steve Gursten", tweet)
    tweet = re.sub(r"MichiganAutoLaw", "Michigan Auto Law", tweet)
    tweet = re.sub(r"birdgang", "bird gang", tweet)
    tweet = re.sub(r"nflnetwork", "NFL Network", tweet)
    tweet = re.sub(r"NYDNSports", "NY Daily News Sports", tweet)
    tweet = re.sub(r"RVacchianoNYDN", "Ralph Vacchiano NY Daily News", tweet)
    tweet = re.sub(r"EdmontonEsks", "Edmonton Eskimos", tweet)
    tweet = re.sub(r"david_brelsford", "David Brelsford", tweet)
    tweet = re.sub(r"TOI_India", "The Times of India", tweet)
    tweet = re.sub(r"hegot", "he got", tweet)
    tweet = re.sub(r"SkinsOn9", "Skins on 9", tweet)
    tweet = re.sub(r"sothathappened", "so that happened", tweet)
    tweet = re.sub(r"LCOutOfDoors", "LC Out Of Doors", tweet)
    tweet = re.sub(r"NationFirst", "Nation First", tweet)
    tweet = re.sub(r"IndiaToday", "India Today", tweet)
    tweet = re.sub(r"HLPS", "helps", tweet)
    tweet = re.sub(r"HOSTAGESTHROSW", "hostages throw", tweet)
    tweet = re.sub(r"SNCTIONS", "sanctions", tweet)
    tweet = re.sub(r"BidTime", "Bid Time", tweet)
    tweet = re.sub(r"crunchysensible", "crunchy sensible", tweet)
    tweet = re.sub(r"RandomActsOfRomance", "Random acts of romance", tweet)
    tweet = re.sub(r"MomentsAtHill", "Moments at hill", tweet)
    tweet = re.sub(r"eatshit", "eat shit", tweet)
    tweet = re.sub(r"liveleakfun", "live leak fun", tweet)
    tweet = re.sub(r"SahelNews", "Sahel News", tweet)
    tweet = re.sub(r"abc7newsbayarea", "ABC 7 News Bay Area", tweet)
    tweet = re.sub(r"facilitiesmanagement", "facilities management", tweet)
    tweet = re.sub(r"facilitydude", "facility dude", tweet)
    tweet = re.sub(r"CampLogistics", "Camp logistics", tweet)
    tweet = re.sub(r"alaskapublic", "Alaska public", tweet)
    tweet = re.sub(r"MarketResearch", "Market Research", tweet)
    tweet = re.sub(r"AccuracyEsports", "Accuracy Esports", tweet)
    tweet = re.sub(r"TheBodyShopAust", "The Body Shop Australia", tweet)
    tweet = re.sub(r"yychail", "Calgary hail", tweet)
    tweet = re.sub(r"yyctraffic", "Calgary traffic", tweet)
    tweet = re.sub(r"eliotschool", "eliot school", tweet)
    tweet = re.sub(r"TheBrokenCity", "The Broken City", tweet)
    tweet = re.sub(r"OldsFireDept", "Olds Fire Department", tweet)
    tweet = re.sub(r"RiverComplex", "River Complex", tweet)
    tweet = re.sub(r"fieldworksmells", "field work smells", tweet)
    tweet = re.sub(r"IranElection", "Iran Election", tweet)
    tweet = re.sub(r"glowng", "glowing", tweet)
    tweet = re.sub(r"kindlng", "kindling", tweet)
    tweet = re.sub(r"riggd", "rigged", tweet)
    tweet = re.sub(r"slownewsday", "slow news day", tweet)
    tweet = re.sub(r"MyanmarFlood", "Myanmar Flood", tweet)
    tweet = re.sub(r"abc7chicago", "ABC 7 Chicago", tweet)
    tweet = re.sub(r"copolitics", "Colorado Politics", tweet)
    tweet = re.sub(r"AdilGhumro", "Adil Ghumro", tweet)
    tweet = re.sub(r"netbots", "net bots", tweet)
    tweet = re.sub(r"byebyeroad", "bye bye road", tweet)
    tweet = re.sub(r"massiveflooding", "massive flooding", tweet)
    tweet = re.sub(r"EndofUS", "End of United States", tweet)
    tweet = re.sub(r"35PM", "35 PM", tweet)
    tweet = re.sub(r"greektheatrela", "Greek Theatre Los Angeles", tweet)
    tweet = re.sub(r"76mins", "76 minutes", tweet)
    tweet = re.sub(r"publicsafetyfirst", "public safety first", tweet)
    tweet = re.sub(r"livesmatter", "lives matter", tweet)
    tweet = re.sub(r"myhometown", "my hometown", tweet)
    tweet = re.sub(r"tankerfire", "tanker fire", tweet)
    tweet = re.sub(r"MEMORIALDAY", "memorial day", tweet)
    tweet = re.sub(r"MEMORIAL_DAY", "memorial day", tweet)
    tweet = re.sub(r"instaxbooty", "instagram booty", tweet)
    tweet = re.sub(r"Jerusalem_Post", "Jerusalem Post", tweet)
    tweet = re.sub(r"WayneRooney_INA", "Wayne Rooney", tweet)
    tweet = re.sub(r"VirtualReality", "Virtual Reality", tweet)
    tweet = re.sub(r"OculusRift", "Oculus Rift", tweet)
    tweet = re.sub(r"OwenJones84", "Owen Jones", tweet)
    tweet = re.sub(r"jeremycorbyn", "Jeremy Corbyn", tweet)
    tweet = re.sub(r"paulrogers002", "Paul Rogers", tweet)
    tweet = re.sub(r"mortalkombatx", "Mortal Kombat X", tweet)
    tweet = re.sub(r"mortalkombat", "Mortal Kombat", tweet)
    tweet = re.sub(r"FilipeCoelho92", "Filipe Coelho", tweet)
    tweet = re.sub(r"OnlyQuakeNews", "Only Quake News", tweet)
    tweet = re.sub(r"kostumes", "costumes", tweet)
    tweet = re.sub(r"YEEESSSS", "yes", tweet)
    tweet = re.sub(r"ToshikazuKatayama", "Toshikazu Katayama", tweet)
    tweet = re.sub(r"IntlDevelopment", "Intl Development", tweet)
    tweet = re.sub(r"ExtremeWeather", "Extreme Weather", tweet)
    tweet = re.sub(r"WereNotGruberVoters", "We are not gruber voters", tweet)
    tweet = re.sub(r"NewsThousands", "News Thousands", tweet)
    tweet = re.sub(r"EdmundAdamus", "Edmund Adamus", tweet)
    tweet = re.sub(r"EyewitnessWV", "Eye witness WV", tweet)
    tweet = re.sub(r"PhiladelphiaMuseu", "Philadelphia Museum", tweet)
    tweet = re.sub(r"DublinComicCon", "Dublin Comic Con", tweet)
    tweet = re.sub(r"NicholasBrendon", "Nicholas Brendon", tweet)
    tweet = re.sub(r"Alltheway80s", "All the way 80s", tweet)
    tweet = re.sub(r"FromTheField", "From the field", tweet)
    tweet = re.sub(r"NorthIowa", "North Iowa", tweet)
    tweet = re.sub(r"WillowFire", "Willow Fire", tweet)
    tweet = re.sub(r"MadRiverComplex", "Mad River Complex", tweet)
    tweet = re.sub(r"feelingmanly", "feeling manly", tweet)
    tweet = re.sub(r"stillnotoverit", "still not over it", tweet)
    tweet = re.sub(r"FortitudeValley", "Fortitude Valley", tweet)
    tweet = re.sub(r"CoastpowerlineTramTr", "Coast powerline", tweet)
    tweet = re.sub(r"ServicesGold", "Services Gold", tweet)
    tweet = re.sub(r"NewsbrokenEmergency", "News broken emergency", tweet)
    tweet = re.sub(r"Evaucation", "evacuation", tweet)
    tweet = re.sub(r"leaveevacuateexitbe", "leave evacuate exit be", tweet)
    tweet = re.sub(r"P_EOPLE", "PEOPLE", tweet)
    tweet = re.sub(r"Tubestrike", "tube strike", tweet)
    tweet = re.sub(r"CLASS_SICK", "CLASS SICK", tweet)
    tweet = re.sub(r"localplumber", "local plumber", tweet)
    tweet = re.sub(r"awesomejobsiri", "awesome job siri", tweet)
    tweet = re.sub(r"PayForItHow", "Pay for it how", tweet)
    tweet = re.sub(r"ThisIsAfrica", "This is Africa", tweet)
    tweet = re.sub(r"crimeairnetwork", "crime air network", tweet)
    tweet = re.sub(r"KimAcheson", "Kim Acheson", tweet)
    tweet = re.sub(r"cityofcalgary", "City of Calgary", tweet)
    tweet = re.sub(r"prosyndicate", "pro syndicate", tweet)
    tweet = re.sub(r"660NEWS", "660 NEWS", tweet)
    tweet = re.sub(r"BusInsMagazine", "Business Insurance Magazine", tweet)
    tweet = re.sub(r"wfocus", "focus", tweet)
    tweet = re.sub(r"ShastaDam", "Shasta Dam", tweet)
    tweet = re.sub(r"go2MarkFranco", "Mark Franco", tweet)
    tweet = re.sub(r"StephGHinojosa", "Steph Hinojosa", tweet)
    tweet = re.sub(r"Nashgrier", "Nash Grier", tweet)
    tweet = re.sub(r"NashNewVideo", "Nash new video", tweet)
    tweet = re.sub(r"IWouldntGetElectedBecause", "I would not get elected because", tweet)
    tweet = re.sub(r"SHGames", "Sledgehammer Games", tweet)
    tweet = re.sub(r"bedhair", "bed hair", tweet)
    tweet = re.sub(r"JoelHeyman", "Joel Heyman", tweet)
    tweet = re.sub(r"viaYouTube", "via YouTube", tweet)
           
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ')      
        
    # Acronyms
    tweet = re.sub(r"MH370", "Malaysia Airlines Flight 370", tweet)
    tweet = re.sub(r"mÌ¼sica", "music", tweet)
    tweet = re.sub(r"okwx", "Oklahoma City Weather", tweet)
    tweet = re.sub(r"arwx", "Arkansas Weather", tweet)    
    tweet = re.sub(r"gawx", "Georgia Weather", tweet)  
    tweet = re.sub(r"scwx", "South Carolina Weather", tweet)  
    tweet = re.sub(r"cawx", "California Weather", tweet)
    tweet = re.sub(r"tnwx", "Tennessee Weather", tweet)
    tweet = re.sub(r"azwx", "Arizona Weather", tweet)  
    tweet = re.sub(r"alwx", "Alabama Weather", tweet)
    tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)    
    tweet = re.sub(r"usNWSgov", "United States National Weather Service", tweet)
    tweet = re.sub(r"Suruc", "Sanliurfa", tweet)   
    
    # Grouping same words without embeddings
    tweet = re.sub(r"Bestnaijamade", "bestnaijamade", tweet)
    tweet = re.sub(r"SOUDELOR", "Soudelor", tweet)
    return tweet

# clean file
def cleanDFs ():
    
    global TRAIN_ROW_COUNT
    
    text_cleaner_transformer = Text_cleaner_transformer ()
    files = ["Data/test.tsv"]  # ["Data/all.tsv", "Data/train.tsv", "Data/test.tsv"]
    for file in files:
        
        df = pd.read_csv (file, sep='\t')
        df['text_cleaned'] = df['text'].apply(lambda s : cleanTweet (s))
        if file != 'Data/test.tsv':
            
            df['target_relabeled'] = df['target'].copy() 
            df.loc[df['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
            df.loc[df['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
            df.loc[df['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
            df.loc[df['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
            df.loc[df['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
            df.loc[df['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
            df.loc[df['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
            df.loc[df['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
            df.loc[df['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
            df.loc[df['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
            df.loc[df['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
            df.loc[df['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
            df.loc[df['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
            df.loc[df['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
            df.loc[df['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
            df.loc[df['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
            df.loc[df['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
            df.loc[df['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0
        
        if file == 'Data/test.tsv':
                df['target_relabeled'] = df['target']
        arrangedDF1 = df[pd.isnull (df['target_relabeled']) == False]
        arrangedDF2 = df[pd.isnull (df['target_relabeled'])]
        df  = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        df.to_csv (file, index=False, sep='\t')
        if file == "Data/all.tsv":
            TRAIN_ROW_COUNT = arrangedDF1.shape[0]
            print ('TRAIN_ROW_COUNT =', TRAIN_ROW_COUNT)
    return

In [ ]:
# file = "Data/train.tsv"
# df = pd.read_csv (file, sep='\t')
# df['target_relabeled'] = df['target'].copy()
# arrangedDF1 = df[pd.isnull (df['target_relabeled']) == False]
# arrangedDF2 = df[pd.isnull (df['target_relabeled'])]
# df  = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
# df.to_csv (file, index=False, sep='\t')
# if file == "Data/all.tsv":
#     TRAIN_ROW_COUNT = arrangedDF1.shape[0]
# print ('TRAIN_ROW_COUNT =', TRAIN_ROW_COUNT)

In [ ]:
def add_meta_features ():
    
    files = ["Data/all.csv", "Data/train.csv", "Data/test.csv"]
    for file in files:
        
        df = pd.read_csv (file)
        
        # word_count
        df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

        # unique_word_count
        df['unique_word_count'] = df['text'].apply(lambda x: len(set(str(x).split())))

        # stop_word_count
        df['stop_word_count'] = df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

        # url_count
        df['url_count'] = df['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

        # mean_word_length
        df['mean_word_length'] = df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

        # char_count
        df['char_count'] = df['text'].apply(lambda x: len(str(x)))

        # punctuation_count
        df['punctuation_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

        # hashtag_count
        df['hashtag_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

        # mention_count
        df['mention_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
        
        file = os.path.splitext (file)[0] + '.tsv'
        df.to_csv (file, index=False, sep='\t')
        print ('Done MetaInfo for', file)
    return
   
def add_bert_features ():
    
    files = ["Data/test.tsv"]   # ["Data/all.tsv", "Data/train.tsv", "Data/test.tsv"]
    for file in files:
        
        print ('Starting Bert for', file)
        df = pd.read_csv (file, sep='\t')
        # bert
        bc = BertClient ()
        bertMatrix = bc.encode (list(df['text_cleaned']))
        bertDF = pd.DataFrame (data=bertMatrix, columns=['bert_'+str (i) for i in range (bertMatrix.shape[1])])
        df = pd.concat ((df, bertDF), axis=1)
        df.to_csv (file, index=False, sep='\t')
        print ('Done Bert for', file)
    return

In [ ]:
# add_meta_features ()
# cleanDFs ()
# add_bert_features ()
# main (threshold=0.9, maxIterCount=1)
# roc_for_clf (BEST_CLF)

# Tune FCNN

In [ ]:
from torch import nn
from skorch import NeuralNetClassifier

class MyModule (nn.Module):
    
    def __init__(self, inputCount=1024, outputCount=2, hiddenLayerCounts=[128], drop_prob=0.2, nonlin=nn.ReLU ()):
        
        super(MyModule, self).__init__()

        self.nonlin  = nonlin
        self.dropout = nn.Dropout (drop_prob)
        
        self.dense1     = nn.Linear (inputCount, hiddenLayerCounts[0])
        self.batchnorm1 = nn.BatchNorm1d (hiddenLayerCounts[0])
        # self.dense2     = nn.Linear(hiddenLayerCounts[0], hiddenLayerCounts[1])
        # self.batchnorm2 = nn.BatchNorm1d (hiddenLayerCounts[1])
        # self.dense3     = nn.Linear(hiddenLayerCounts[1], hiddenLayerCounts[2])
        # self.batchnorm3 = nn.BatchNorm1d (hiddenLayerCounts[2])        
        self.outDense   = nn.Linear (hiddenLayerCounts[-1], outputCount)
        
        self.outActivtn = None
        if outputCount == 1 or outputCount == 2:
            self.outActivtn = nn.Sigmoid ()
        else:
            self.outActivtn = nn.Softmax (dim=-1)
        return

    def forward (self, X, **kwargs):
        
        X = self.dropout (self.nonlin (self.batchnorm1 (self.dense1 (X))))
        # X = self.dropout (self.nonlin (self.batchnorm2 (self.dense2 (X))))
        # X = self.dropout (self.nonlin (batchnorm3 (self.dense3 (X))))
        X = self.outActivtn (self.outDense (X))
        return X

In [ ]:
FT = bert_meta_keyword_ica_ft

def get_X_for_FCNN ():
    
    global trainDF, evalDF, TRAIN_ROW_COUNT, bert_meta_ft, FT
    TRAIN_ROW_COUNT = 8234
    
    train_file = "Data/all.tsv"
    rawDF = pd.read_csv (train_file, sep='\t')  #, dtype={'id': np.int16, 'target_relabeled': np.int8})
    # rawDF = rawDF[0:30]               # Comment this out ********************************
    
    labeledDF = rawDF[:TRAIN_ROW_COUNT]
    trainDF, evalDF = train_test_split (labeledDF, test_size=0.33, random_state=42)
    trainDF_temp = pd.concat ((trainDF, rawDF[TRAIN_ROW_COUNT:]), axis=0)
    TRAIN_ROW_COUNT = trainDF.shape[0]
    trainDF = trainDF_temp
    
    print ('rawDF.shape =', rawDF.shape, 'trainDF.shape =', trainDF.shape, 'evalDF.shape =', evalDF.shape)
    rawDF, labeledDF, trainDF_temp = None, None, None
    
    X = FT.fit_transform (trainDF, trainDF['target_relabeled'])
    Y = trainDF['target_relabeled']
    X = X[:TRAIN_ROW_COUNT, :]
    Y = Y[:TRAIN_ROW_COUNT]    
    X = X.astype (np.float32)
    Y = Y.astype (np.int64)
    print ('X.shape =', X.shape)
    return X, Y

In [ ]:
import matplotlib.pyplot as plt

def tune_FCNN ():
    
    global FT
    
    X, Y = get_X_for_FCNN ()
    CV_SCORERS = {
        'precision_score':         make_scorer (precision_score),
        'recall_score':            make_scorer (recall_score),
        'f1_score':                make_scorer (f1_score),
        'balanced_accuracy_score': make_scorer (balanced_accuracy_score)
    }
    cv_scores = []
    balanced_accs = []
    param_grid = {
        # 'lr': [0.01, 0.005],
        # 'max_epochs': [30, 80],
        'module__hiddenLayerCounts': [[5], [10], [20]]
    }
    lrScheduler = LRScheduler (
        CyclicLR,
        base_lr=0.0001,
        max_lr=0.05,
        step_every='batch'
    )
    clf = NeuralNetClassifier (

            module=MyModule,
            module__inputCount=X.shape[1], 
            module__outputCount=2, 
            # module__hiddenLayerCounts=[hidden_count],
            max_epochs=1000,
            # lr=0.01,
            iterator_train__shuffle=True,
            callbacks=[('LRScheduler', lrScheduler), ('EarlyStopping', EarlyStopping (patience=20))]
    )
    gridSearchCV = GridSearchCV (
        clf, param_grid=param_grid, iid=False, cv=4, scoring=CV_SCORERS, 
        refit='f1_score'
    )
    gridSearchCV.fit (X, Y)
    clf = gridSearchCV.best_estimator_
    cv_score = gridSearchCV.best_score_ 
    print ('Best CV params =', gridSearchCV.best_params_)
    hiddenLayerCounts = gridSearchCV.best_params_['module__hiddenLayerCounts']
    # clf.fit (X, Y) # not required

    X_eval = FT.transform (evalDF)
    X_eval = X_eval.astype (np.float32)
    Y_eval = evalDF['target_relabeled']
    Y_eval = Y_eval.astype (np.int64)
    predictions = clf.predict (X_eval).astype (np.int64)
    f1_acc = f1_score(Y_eval, predictions)
    print ('------------------------- cv_score =', cv_score, 'f1_score =', f1_acc, '-------------------------')

    # plt.plot (hidden_counts, cv_scores, 'bs', hidden_counts, balanced_accs, 'g^')
    # plt.show ()
    return hiddenLayerCounts

In [ ]:
HIDDEN_LayerCounts = 10
# HIDDEN_LayerCounts = tune_FCNN ()

# Findings

100 iterations, lr=0.01:

bert + meta:
    5 : cv_score = 0.7493871012282531 f1_score = 0.7361596009975062  *
    10: cv_score = 0.7222967261625052 f1_score = 0.7465753424657535
    20: cv_score = 0.7454181475969770 f1_score = 0.7130712008501594
    40: cv_score = 0.7328312987036163 f1_score = 0.7467312348668281

bert + meta + keyword:
    5 : cv_score = 0.7496093435110001 f1_score = 0.7058823529411763
    10: cv_score = 0.7441278912831047 f1_score = 0.7675417661097851  *
    20: cv_score = 0.7357580671447163 f1_score = 0.7607768829938418
    40: cv_score = 0.7505499734217471 f1_score = 0.7597189695550353
    80: cv_score = 0.7519284826539710 f1_score = 0.7247081712062258

bert + meta + keyword + cosineSimRatio
    5 : cv_score = 0.7562631511302100 f1_score = 0.7270101107366395
    10: cv_score = 0.7396842074042981 f1_score = 0.7367901234567901
    20: cv_score = 0.7555392927016631 f1_score = 0.7443682664054848
    40: cv_score = 0.7606594232531072 f1_score = 0.7557003257328989  *
    
bert + meta + keyword + cosineSimRatio + ICA:
    5 : cv_score = 0.6739008093714172 f1_score = 0.6790123456790124
    10: cv_score = 0.7522109377868911 f1_score = 0.7671357240127098  *
    20: cv_score = 0.7474259878739999 f1_score = 0.7145152651312265
    40: cv_score = 0.7546149099255375 f1_score = 0.7488986784140969 
    80: cv_score = 0.7523111740532332 f1_score = 0.7439803439803440
    Final F1 = 0.7376456408196062
    
bert + meta + keyword:
    Best CV params = {'max_epochs': 80, 'module__hiddenLayerCounts': [20]}
    Discrete2OneHot_FeatureTransformer: transform(): X.shape = (2721, 222)
    ------------------------- cv_score = 0.7497306991236068 f1_score = 0.7628341111582521 
    Final F1 = 0.7546099290780142

bert:
    Best CV params = {'module__hiddenLayerCounts': [20]}
    ------------------------- cv_score = 0.76457992604136 f1_score = 0.760537285780454 -------------------------
    Final F1 = 0.7514

In [ ]:
def predict_FCNN ():
    
    global trainDF, evalDF, TRAIN_ROW_COUNT, FT, HIDDEN_LayerCounts
    TRAIN_ROW_COUNT = 8234
    
    train_file = "Data/all.tsv"
    rawDF = pd.read_csv (train_file, sep='\t')
    # rawDF = rawDF[0:30]               # Comment this out ********************************
    trainDF = rawDF[:TRAIN_ROW_COUNT]
    rawDF = None
    
    X = FT.fit_transform (trainDF, trainDF['target_relabeled'])
    Y = trainDF['target_relabeled']
    X = X[:TRAIN_ROW_COUNT, :]
    Y = Y[:TRAIN_ROW_COUNT]    
    X = X.astype (np.float32)
    Y = Y.astype (np.int64)
    print ('train: X.shape =', X.shape)
    
    lrScheduler = LRScheduler (
        
        CyclicLR,
        base_lr=0.0001,
        max_lr=0.05,
        step_every='batch'
    )
    clf = NeuralNetClassifier (

            module=MyModule,
            module__inputCount=X.shape[1], 
            module__outputCount=2, 
            module__hiddenLayerCounts=HIDDEN_LayerCounts,
            max_epochs=1000,
            # lr=0.01,
            iterator_train__shuffle=True,
            callbacks=[('LRScheduler', lrScheduler), ('EarlyStopping', EarlyStopping (patience=20))]
    )
    clf.fit (X, Y)
    
    test_file = "Data/test.tsv"
    testDF = pd.read_csv (test_file, sep='\t')
    X = FT.transform (testDF)
    X = X.astype (np.float32)
    print ('test: X.shape =', X.shape)
    Y = np.round (clf.predict (X)).astype (int)
    Y_perfect = testDF['target'].astype (int)
    f1_acc = f1_score (Y_perfect, Y)
    print ('\n************** F1-Score =', f1_acc, "*************\n")
    
    submitDF = testDF[['id', 'target']]
    submitDF['target'] = Y
    submitDF = submitDF.astype (int)
    submitDF.to_csv ('submission_v'+str (VERSION) + 'FCNN.csv', index=False)
    print ("Done Prediction !")

In [ ]:
# predict_FCNN ()